# OpenACC Loop Optimizations

此实验面向 C/C++ 程序员。如果您更喜欢使用 Fortran，请单击 [此链接](../Fortran/README.ipynb)

不要忘记查看其他 [OpenACC 资源](https://www.openacc.org/resources) 并加入我们的 [OpenACC Slack 频道](https://www.openacc.org/community#slack)，以分享您的经验并从社区获得更多帮助。

---

让我们执行下面的单元格以显示有关服务器上运行的 GPU 的信息。为此，请执行下面的单元格块，方法是将焦点放在它上面（用鼠标单击它），然后按 Ctrl-Enter，或按上面工具栏中的播放按钮。如果一切顺利，您应该会看到灰色单元格下方返回一些输出。

In [ ]:
!nvidia-smi

---

## 介绍

该实验室的目标是使用 OpenACC 循环条款来融合我们的并行循环。

![development_cycle.png](../images/development_cycle.png)

这是 OpenACC 3 步开发周期。

**分析**您的代码，并预测可以在哪里发现潜在的并行性。使用 Profiler 来帮助了解代码中正在发生的事情以及并行性可能存在的地方。

**并行**您的代码，从最耗时的零件开始。专注于保持程序的正确结果。

**优化**您的代码，专注于最大化性能。在早期并行化期间，性能可能不会增加。

我们目前正在解决 **Optimize** 步骤。我们将包括 OpenACC 循环条款，以优化并行循环巢的执行。

---

## 运行代码

在以前的实验室中，我们建立了一个工作的并行代码，该代码可以在多核心 CPU 和 GPU 上运行。让我们运行代码并注意性能，以便我们可以将运行时与我们进行的任何未来优化进行比较。

In [ ]:
!nvc -fast -ta=tesla -Minfo=accel -o laplace_baseline jacobi.c laplace2d.c && ./laplace_baseline

### 可选：查看代码

如果您想复习一下我们正在处理的代码文件，您可以使用下面的两个链接查看它们。

[jacobi.c](jacobi.c)

[laplace2d.c](laplace2d.c)

### 可选：分析代码

In [ ]:
!nsys profile -t openacc --stats=true --force-overwrite true -o laplace_baseline ./laplace_baseline

让我们查看一下分析器的报告。分析运行完成后，按住 <mark>Shift</mark> 并<mark>右键单击</mark> [此处](laplace_baseline.qdrep)（选择 *将链接另存为*）下载并保存报告文件，然后通过 GUI 打开它。要在本地查看分析器报告，请参阅 [如何查看报告](../../../module2/assets/C/README.ipynb) 部分。

---

## OpenACC 循环指令

*循环指令*允许我们标记特定循环以进行并行化。*循环指令*还允许我们使用**循环子句**将特定优化/更改映射到我们的循环。并非所有循环子句都用于优化我们的代码；有些还用于验证代码的正确性。*循环指令*的一些示例如下：

```cpp
#pragma acc parallel loop < loop clauses >
for(int i = 0; i < N; i++)
{
    < loop code >
}
```

```cpp
#pragma acc kernels loop < loop clauses >
for(int i = 0; i < N; i++)
{
    < loop code >
}
```

```cpp
#pragma acc parallel loop < loop clauses >
for(int i = 0; i < N; i++)
{
    #pragma acc loop < loop clauses >
    for(int j = 0; j < M; j++)
    {
        < loop code >
    }
}
```

此外，包括循环优化并不总是能优化代码。程序员需要通过分析和分析来确定哪些循环优化最适合他们的循环。

### Independent 子句

使用 `kernels` 指令时，编译器将决定哪些循环可并行化，哪些不可并行化。但是，程序员可以使用 `independent` 子句来覆盖此编译器决定。`independent` 子句是程序员向编译器保证循环可并行化的一种方式。

```cpp
#pragma acc kernels loop independent
for(int i = 0; i < N; i++)
{
    < Parallel Loop Code >
}

#pragma acc kernels
{
    for(int i = 0; i < N; i++)
    {
        < Parallel Loop Code >
    }
    
    #pragma acc loop independent
    for(int i = 0; i < N; i++)
    {
        < Parallel Loop Code >
    }
}
```

在第二个示例中，我们有两个循环。编译器将决定第一个循环是否可并行化。然而，在第二个循环中，我们包含了 independent 子句。这意味着编译器将信任程序员，并假设第二个循环可并行化。

使用 `parallel` 指令时，`independent` 子句将自动隐含在其中包含的所有 `loop` 指令中。这意味着当您使用 `parallel` 指令时，您无需使用 `independent` 子句。

### Auto 子句

`auto` 子句与 `independent` 子句基本完全相反。使用 `parallel` 指令时，编译器将信任程序员决定的任何内容。这意味着，如果程序员认为循环是可并行的，编译器将信任程序员。但是，如果在循环中包含 auto 子句，编译器将仔细检查循环，并决定是否对其进行并行化。

```cpp
#pragma acc parallel loop auto
for(int i = 0; i < N; i++)
{
    < Parallel Loop Code >
}
```

`independent` 子句是程序员向编译器断言循环可并行化的一种方法。`auto` 子句是程序员告诉编译器分析循环并确定其是否可并行化的一种方法。

### Seq 子句

`seq` 子句（*“sequence”* 的缩写）用于定义应在并行硬件上按顺序运行的循环。此循环子句通常自动应用于大型、多维循环嵌套，因为编译器可能只能描述最外层循环的并行性。例如：

```cpp
for(int i = 0; i < N; i++)
{
    for(int j = 0; j < M; j++)
    {
        for(int k = 0; k < Q; k++)
        {
            < Loop Code >
        }
    }
}
```

编译器可能只能并行化 `i` 和 `j` 循环，并选择按顺序运行 `k` 循环。`seq` 子句对于按顺序运行非常小的嵌套循环也很有用。例如：

```cpp
for(int i = 0; i < 1000000; i++)
{
    for(int j = 0; j < 4; j++)
    {
        for(int k = 0; k < 1000000; k++)
        {
            < Loop Code >
        }
    }
}
```

中间的循环非常小，很可能不会从并行化中受益。为了解决这个问题，我们可以应用 `seq` 子句，如下所示：

```cpp
#pragma acc parallel loop
for(int i = 0; i < 1000000; i++)
{
    #pragma acc loop seq
    for(int j = 0; j < 4; j++)
    {
        #pragma acc loop
        for(int k = 0; k < 1000000; k++)
        {
            < Loop Code >
        }
    }
}
```

在此代码片段中，中间循环将按顺序运行，而最外层循环和最内层循环将并行运行。

### 归约子句

到目前为止，我们在 laplace 代码中一直在使用 `归约` 子句。我们没有明确定义归约，而是编译器自动将归约子句应用于我们的代码。让我们看看 `laplace2d.c` 代码文件中的其中一个循环。

```cpp
#pragma acc parallel loop present(A,Anew)
for( int j = 1; j < n-1; j++
{
    #pragma acc loop
    for( int i = 1; i < m-1; i++ )
    {
        Anew[OFFSET(j, i, m)] = 0.25 * ( A[OFFSET(j, i+1, m)] + A[OFFSET(j, i-1, m)] 
                                       + A[OFFSET(j-1, i, m)] + A[OFFSET(j+1, i, m)] );
        error = fmax( error, fabs(Anew[OFFSET(j, i, m)] - A[OFFSET(j, i , m)]));
    }
}
```

更具体地说，让我们关注这一行代码：

```cpp
error = fmax( error, fabs(Anew[OFFSET(j, i, m)] - A[OFFSET(j, i , m)]));
```

我们的内循环的每次迭代都会写入值 `error` 。当我们**同时**运行数千个这样的循环迭代时，让它们全部直接写入` error` 会变得非常危险。为了解决这个问题，我们必须使用 OpenACC `reduction` 子句。让我们看看语法。

```cpp
#pragma acc parallel loop reduction(operator:value)
```

让我们看一个简单的使用示例。

```cpp
#pragma acc parallel loop reduction(+:sum)
for(int i = 0; i < N; i++)
{
    sum += A[i];
}
```

This is a list of all of the available operators in OpenACC.

|Operator    |Example                     |Description           |
|:----------:|:---------------------------|:---------------------|
|+           |reduction(+:sum)            |Mathematical summation|
|*           |reduction(*:product)        |Mathematical product  |
|max         |reduction(max:maximum)      |Maximum value         |
|min         |reduction(min:minimum)      |Minimum value         |
|&           |reduction(&:val)            |Bitwise AND           |
|&#124;      |reduction(&#124;:val)       |Bitwise OR            |
|&&          |reduction(&&:bool)          |Logical AND           |
|&#124;&#124;|reduction(&#124;&#124;:bool)|Logical OR            |

#### 可选：实现 Reduction 子句

我们使用 nvc 编译器编译代码，该编译器可以自动包含 Reduction 子句。但是，在其他编译器中，我们可能没有那么幸运。使用以下链接将带有 `max` 运算符的 `reduction` 子句添加到我们的代码中。

[laplace2d.c](laplace2d.c)

（请确保使用 ctrl+s 保存代码）

然后，您可以运行以下脚本来验证编译器是否正确识别了您的 Reduction 子句。

In [ ]:
!nvc -ta=tesla -Minfo=accel -o laplace_reduction jacobi.c laplace2d.c && ./laplace_reduction

您也可以通过点击下方的链接来检查您的答案。

[laplace2d.c](solutions/reduction/laplace2d.c)

### Private 子句

private 子句允许我们将某些变量（甚至数组）标记为 *“private”* 。最好的可视化方法是使用以下示例：

```cpp
int tmp;

#pragma acc parallel loop private(tmp)
for(int i = 0; i < N/2; i++)
{
    tmp = A[i];
    A[i] = A[N-i-1];
    A[N-i-1] = tmp;
}
```

在此代码中，每个线程都有自己的 *tmp 的私有副本*。您还可以将静态数组声明为私有的，如下所示：

```cpp
int tmp[10];

#pragma acc parallel loop private(tmp[0:10])
for(int i = 0; i < N; i++)
{
    < Loop code that uses the tmp array >
}
```

使用*私有变量*时，变量仅存在于私有范围内。这通常意味着私有变量仅存在于单个循环迭代中，并且存储在私有变量中的值不能扩展到循环之外。

### 折叠子句

这是我们第一次真正的*循环优化*。`collapse` 子句允许我们将多维循环嵌套转换为单维循环。此过程有助于增加循环的总长度（通常会增加并行性），并且通常有助于内存局部性。让我们看看语法。

```cpp
#pragma acc parallel loop collapse( N )
```

其中 N 是要折叠的循环数。

```cpp
#pragma acc parallel loop collapse( 3 )
for(int i = 0; i < N; i++)
{
    for(int j = 0; j < M; j++)
    {
        for(int k = 0; k < Q; k++)
        {
            < loop code >
        }
    }
}
```

此代码将三维循环嵌套组合成单个一维循环。需要注意的是，使用 `collapse` 子句时，内层循环不应有自己的 `loop` 指令。这意味着以下代码片段是**不正确**的，并且在编译时会发出警告。

```cpp
#pragma acc parallel loop collapse( 3 )
for(int i = 0; i < N; i++)
{
    #pragma acc loop
    for(int j = 0; j < M; j++)
    {
        #pragma acc loop
        for(int k = 0; k < Q; k++)
        {
            < loop code >
        }
    }
}
```

#### 实现折叠子句

使用以下链接编辑我们的代码。使用`collapse` 子句将我们的多维循环折叠为一维循环。

[laplace2d.c](laplace2d.c)

（确保使用 ctrl+s 保存代码）

然后运行以下脚本以查看代码如何运行。

In [ ]:
!nvc -ta=tesla -Minfo=accel -o laplace_collapse jacobi.c laplace2d.c && ./laplace_collapse

### Tile 子句

`tile` 子句允许我们将多维循环分解为 *tiles* 或 *blocks*。这通常有助于增加某些代码中的内存局部性。让我们看看语法。

```cpp
#pragma acc parallel loop tile( x, y, z, ... )
```

我们的图块可以有任意多的尺寸，但我们必须小心不要创建太大的图块。让我们看一个例子：

```cpp
#pragma acc parallel loop tile( 32, 32 )
for(int i = 0; i < N; i++)
{
    for(int j = 0; j < M; j++)
    {
        < loop code >
    }
}
```

上述代码会将循环迭代拆分为 32x32 个图块（或块），然后并行执行这些块。让我们看一个稍微更具体的代码。

```cpp
#pragma acc parallel loop tile( 32, 32 )
for(int i = 0; i < 128; i++)
{
    for(int j = 0; j < 128; j++)
    {
        < loop code >
    }
}
```

在此代码中，我们有 128x128 次循环迭代，它们被分解为 32x32 个图块。这意味着我们将有 16 个图块，每个图块的大小为 32x32。与“collapse”子句类似，内循环不应包含“loop”指令。这意味着以下代码是**不正确的**，并且在编译时会发出警告。

```cpp
#pragma acc parallel loop tile( 32, 32 )
for(int i = 0; i < N; i++)
{
    #pragma acc loop
    for(int j = 0; j < M; j++)
    {
        < loop code >
    }
}
```

#### 实现 Tile 子句

使用以下链接编辑我们的代码。用 `tile` 子句替换 `collapse` 子句，将我们的多维循环分解为更小的图块。尝试使用各种不同的图块大小，但始终将其中一个维度保持为 **32 的倍数**。我们稍后会讨论为什么这很重要。

[laplace2d.c](laplace2d.c)

（确保使用 ctrl+s 保存代码）

然后运行以下脚本以查看代码的运行情况。

In [ ]:
!nvc -ta=tesla -Minfo=accel -o laplace_tile jacobi.c laplace2d.c && ./laplace_tile

### Gang/Worker/Vector

这是我们的最后一个优化，可以说是最重要的一个。在 OpenACC 中，*Gang*、*Worker* 和 *Vector* 的概念用于定义额外的并行级别。具体来说，对于 NVIDIA GPU，*Gang*、*Worker* 和 *Vector* 将定义我们的 GPU 线程的 *组织*（有时称为 for 循环迭代的 *分解*）。每个循环都有一个最佳的 Gang、Worker 和/或 Vector 实现，找到正确的实现通常需要一些思考，甚至可能需要反复试验。所以让我们解释一下 Gang、Worker 和 Vector 的实际工作原理。

![gang_worker_vector.png](../images/gang_worker_vector.png)

此图表示单个**组**。在并行化我们的**for 循环**时，**循环迭代**将在多个组之间**均匀分配**。每个组将包含多个**线程**。这些线程被组织成**块**。**worker** 是一行线程。在上图中，有 3 个**worker**，这意味着有 3 行线程。**vector** 指的是每行的长度。因此，在上图中，vector 为 8，因为每行有 8 个线程长。

默认情况下，在为 GPU 编程时，会自动应用**组**和**vector**并行性。让我们看一个简单的 GPU 示例代码，其中我们明确展示了组和vector 的工作原理。

```cpp
#pragma acc parallel loop gang
for(int i = 0; i < N; i++)
{
    #pragma acc loop vector
    for(int j = 0; j < M; j++)
    {
        < loop code >
    }
}
```

外循环将均匀分布在多个**帮派**中。然后，在这些帮派中，内循环将在**向量**中并行执行。这个过程通常会自动发生，但是，我们通常可以通过自己优化*帮派*、*工作者*和*向量*来实现更好的性能。

让我们看一个例子，其中使用*帮派*、*工作者*和*向量*可以大大增加循环的并行性。

```cpp
#pragma acc parallel loop gang
for(int i = 0; i < N; i++)
{
    #pragma acc loop vector
    for(int j = 0; j < M; k++)
    {
        for(int k = 0; k < Q; k++)
        {
            < loop code >
        }
    }
}
```

在这个循环中，我们在外循环上有**帮派级**并行性，在中间循环上有**向量级**并行性。但是，内循环没有任何并行性。这意味着每个线程都将运行内循环，但是，GPU 线程实际上并不是用来运行整个循环的。为了解决这个问题，我们可以使用**工作者级**并行性来添加另一个层。

```cpp
#pragma acc parallel loop gang
for(int i = 0; i < N; i++)
{
    #pragma acc loop worker
    for(int j = 0; j < M; k++)
    {
        #pragma acc loop vector
        for(int k = 0; k < Q; k++)
        {
            < loop code >
        }
    }
}
```

现在，外循环将跨团伙拆分，中间循环将跨工作者拆分，内循环将由向量内的线程执行。

#### Gang、Worker 和 Vector 语法

到目前为止，我们一直在展示 *Gang*、*Worker* 和 *Vector* 的非常通用的示例。*Gang*、*Worker* 和 *Vector* 的最大优势之一是能够明确定义您需要多少个 gang 和 worker，以及 vector 中应该有多少个线程。让我们看一下 `parallel` 指令的语法：

```cpp
#pragma acc parallel num_gangs( 2 ) num_workers( 4 ) vector_length( 32 )
{
    #pragma acc loop gang worker
    for(int i = 0; i < N; i++)
    {
        #pragma acc loop vector
        for(int j = 0; j < M; j++)
        {
            < loop code >
        }
    }
}
```

现在来看看 `kernels` 指令的语法：

```cpp
#pragma acc kernels loop gang( 2 ) worker( 4 )
for(int i = 0; i < N; i++)
{
    #pragma acc loop vector( 32 )
    for(int j = 0; j < M; j++)
    {
        < loop code >
    }
}
```

#### 避免浪费线程

并行化小数组时，必须注意向量中的线程数不要大于循环迭代次数。让我们看一个简单的例子：

```cpp
#pragma acc kernels loop gang
for(int i = 0; i < 1000000000; i++)
{
    #pragma acc loop vector(256)
    for(int j = 0; j < 32; j++)
    {
        < loop code >
    }
}
```

在此代码中，我们正在并行化一个具有 32 次迭代的内循环。但是，我们的向量长度为 256 个线程。这意味着当我们运行此代码时，我们将拥有比循环迭代多得多的线程，并且许多线程将处于空闲状态。我们可以通过几种不同的方式修复此问题，但让我们使用**工作者级并行性**来修复它。

```cpp
#pragma acc kernels loop gang worker(8)
for(int i = 0; i < 1000000000; i++)
{
    #pragma acc loop vector(32)
    for(int j = 0; j < 32; j++)
    {
        < loop code >
    }
}
```

最初，我们有 1 个（隐含的）工作线程，其中包含 256 个线程。现在，我们有 8 个工作线程，每个工作线程只有 32 个线程。我们通过减少 **vector** 的长度并增加 **worker** 的数量消除了所有浪费的线程。

#### 32 规则（Warps）

NVIDIA GPU 编程的一般经验法则是始终确保向量长度是 32 的倍数（即 32、64、96、128、... 512、... 1024... 等）。这是因为 NVIDIA GPU 针对使用 *warps* 进行了优化。Warps 是执行相同计算机指令的 32 个线程组。因此作为参考：

```cpp
#pragma acc kernels loop gang
for(int i = 0; i < N; i++)
{
    #pragma acc loop vector(32)
    for(int j = 0; j < M; j++)
    {
        < loop code >
    }
}
```

表现将优于：

```cpp
#pragma acc kernels loop gang
for(int i = 0; i < N; i++)
{
    #pragma acc loop vector(31)
    for(int j = 0; j < M; j++)
    {
        < loop code >
    }
}
```

#### 实现 Gang、Worker 和 Vector 子句

使用以下链接编辑我们的代码。用 `gang`、`worker` 和/或 `vector` 子句替换我们之前的子句，以重新组织我们的线程块。尝试使用几个不同的数字，但始终将向量长度保持为 **32 的倍数**，以充分利用 **warps** 。

[laplace2d.c](laplace2d.c)

（确保使用 ctrl+s 保存代码）

然后运行以下脚本以查看代码的运行情况。

In [ ]:
!nvc -ta=tesla -Minfo=accel -o laplace_gang_worker_vector jacobi.c laplace2d.c && ./laplace_gang_worker_vector

## 使用我们学到的一切

现在我们已经介绍了编辑循环的各种方法，将这些知识应用到我们的 laplace 代码中。尝试混合一些循环子句，看看并行和内核指令之间的循环优化有何不同。

您可以运行以下脚本，使用 `kernels` 指令重置您的代码。

In [ ]:
!cp ./solutions/kernels/laplace2d.c ./laplace2d.c && echo "Reset Finished"

您可以运行以下脚本，使用 `parallel` 指令重置您的代码。

In [ ]:
!cp ./solutions/parallel/laplace2d.c ./laplace2d.c && echo "Reset Finished"

然后使用以下链接编辑我们的 laplace 代码。

[laplace2d.c](laplace2d.c)

（确保使用 ctrl+s 保存代码）

然后运行以下脚本以查看代码如何运行。

In [ ]:
!nvc -ta=tesla -Minfo=accel -o laplace jacobi.c laplace2d.c && ./laplace

---

## 总结

使用 OpenACC 的主要目标是并行化大型 for 循环。为了实现这一点，我们必须使用 OpenACC *loop 指令* 和 *loop 子句*。有很多方法可以改变和优化我们的循环，但这取决于程序员通过分析和分析来确定哪种方法最好。在实验室系列的这一点上，您应该能够开始并行化您自己的个人代码，并能够使用 OpenACC 实现相对较高的性能。

---

## 额外任务

如果您想要一些关于使用 OpenACC 的额外课程，OpenACC YouTube 页面上有一个 OpenACC 简介视频系列。如果您还没有看过，我建议您观看这个 6 部分系列。每个视频不到 10 分钟，将以视觉和亲身体验的方式展示我们在这些实验室中涵盖的大量材料。以下链接将带您进入该系列的第 1 部分。

[OpenACC 并行编程简介 - 第 1 部分](https://youtu.be/PxmvTsrCTZg)

---

## 实验后总结

如果您想下载此实验以供以后查看，建议您转到浏览器的文件菜单（而不是 Jupyter 笔记本文件菜单）并保存完整的网页。这将确保图像也被复制下来。

您还可以执行以下单元块来创建您正在处理的文件的 zip 文件，并使用下面的链接下载它。

In [ ]:
%%bash
rm -f openacc_files.zip
zip -r openacc_files.zip *

**执行上述 zip 命令后**，您应该能够通过按住 <mark>Shift</mark> 并<mark>右键单击</mark> [此处](openacc_files.zip) 并选择*将链接另存为*来下载并保存 zip 文件。

# 许可

本材料由 NVIDIA Corporation 根据知识共享署名 4.0 国际 (CC BY 4.0) 发布。